In [ ]:
import sys

sys.path.append('..')

from db import *
from analysis import *
from blackbox.helpers import *

# DALL-E-3 Low-Resource Language Attack

In [ ]:
data = (
    session.query(
        Languages.name.label("Language"),
        Languages.common_crawl_pages.label("Common Crawl Pages"),
        func.count(ImageCreationRequest.id).label("Total Requests"),
        func.count(case((ImageCreationRequest.success == True, 1))).label("Success Count"),
        func.count(case((ImageCreationRequest.success == False, 1))).label("Failure Count"),
        func.coalesce(
            func.count(case((ImageCreationRequest.success == True, 1))) * 100.0 /
            func.count(ImageCreationRequest.id), 0
        ).label("Acceptance Rate (%)"),
    )
    .join(TranslatedPrompts, ImageCreationRequest.translated_prompt_id == TranslatedPrompts.id)
    .join(Languages, TranslatedPrompts.language_id == Languages.id)
    .join(Prompts, TranslatedPrompts.prompt_id == Prompts.id)
    .join(PromptPrefixStyles, ImageCreationRequest.prompt_prefix_id == PromptPrefixStyles.id)
    .filter(
        ImageCreationRequest.model == 'dalle-3',
        Prompts.dataset == 'baseline'
    )
    .group_by(Languages.id, Languages.name_short, Languages.common_crawl_pages)
    .order_by(Languages.common_crawl_pages.desc())
)

df = query_to_df(data)

# Results

**Description:** This experiment demonstrates that low-resource languages can be used as an effective mechanism to bypass DALL·E safety filters (§6.1).

**Dataset:** This experiment uses 5 prompts in 5 languages (English, Nepali, Sinhala, Lao, Hawaiian) for a total of 25 requests to DALL·E 3.

**Success Criteria:** Languages with fewer pages in the Common Crawl dataset often demonstrating higher prompt acceptance rates.

In [ ]:
df